In [7]:
import onnx
# 필요한 import문
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

batch_size = 1
input_SequenceLength = 5
x = torch.randn(input_SequenceLength, batch_size, 3, 224, 224, requires_grad=True)
# x = torch.randn(3, 224, 224, requires_grad=True)
onnx_model = onnx.load("KSLmodel_resnet50.onnx")
onnx.checker.check_model(onnx_model)

In [8]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("KSLmodel_resnet50.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# ONNX 런타임에서 계산된 결과값
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
# print(ort_inputs.values())
ort_outs = ort_session.run(None, ort_inputs)

# # ONNX 런타임과 PyTorch에서 연산된 결과값 비교
# print("Exported model has been tested with ONNXRuntime, and the result looks good!")

In [9]:
from PIL import Image
import torchvision.transforms as transforms

img = Image.open("/opt/project/cat_224x224.jpg")

resize = transforms.Resize([224, 224])
img = resize(img)

img_ycbcr = img.convert('YCbCr')
img_y, img_cb, img_cr = img_ycbcr.split()

to_tensor = transforms.ToTensor()

img_y = to_tensor(img)
img_y.unsqueeze_(0)
print(img_y.shape)
img = torch.stack([img_y,img_y], dim=0)
img = torch.cat([img,img], dim=0)

print(img.shape)

torch.Size([1, 3, 224, 224])
torch.Size([4, 1, 3, 224, 224])


In [10]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]
print(ort_outs[0])

[[0.0225473  0.11896955 0.08581965 0.08800738 0.07835913 0.15181457
  0.16795596 0.07015321 0.21637318]]


In [11]:
# img_out_y = Image.fromarray(np.uint8((img_out_y[0] * 255.0).clip(0, 255)[0]), mode='L')

# # PyTorch 버전의 후처리 과정 코드를 이용해 결과 이미지 만들기
# final_img = Image.merge(
#     "YCbCr", [
#         img_out_y,
#         img_cb.resize(img_out_y.size, Image.BICUBIC),
#         img_cr.resize(img_out_y.size, Image.BICUBIC),
#     ]).convert("RGB")

# # 이미지를 저장하고 모바일 기기에서의 결과 이미지와 비교하기
# final_img.save("/home/khs/Documents/final_proj/cat_224x224_2.jpg")

In [17]:
import tensorrt

In [22]:
!trtexec --onnx=./KSLmodel_resnet50.onnx --saveEngine=KSLmodel_resnet50.trt  --explicitBatch --inputIOFormats=fp16:chw --outputIOFormats=fp16:chw --fp16



&&&& RUNNING TensorRT.trtexec # trtexec --onnx=./KSLmodel_resnet50.onnx --saveEngine=KSLmodel_resnet50.trt --explicitBatch --inputIOFormats=fp16:chw --outputIOFormats=fp16:chw --fp16
[11/27/2023-14:34:52] [I] === Model Options ===
[11/27/2023-14:34:52] [I] Format: ONNX
[11/27/2023-14:34:52] [I] Model: ./KSLmodel_resnet50.onnx
[11/27/2023-14:34:52] [I] Output:
[11/27/2023-14:34:52] [I] === Build Options ===
[11/27/2023-14:34:52] [I] Max batch: explicit
[11/27/2023-14:34:52] [I] Workspace: 16 MiB
[11/27/2023-14:34:52] [I] minTiming: 1
[11/27/2023-14:34:52] [I] avgTiming: 8
[11/27/2023-14:34:52] [I] Precision: FP32+FP16
[11/27/2023-14:34:52] [I] Calibration: 
[11/27/2023-14:34:52] [I] Refit: Disabled
[11/27/2023-14:34:52] [I] Safe mode: Disabled
[11/27/2023-14:34:52] [I] Save engine: KSLmodel_resnet50.trt
[11/27/2023-14:34:52] [I] Load engine: 
[11/27/2023-14:34:52] [I] Builder Cache: Enabled
[11/27/2023-14:34:52] [I] NVTX verbosity: 0
[11/27/2023-14:34:52] [I] Tactic sources: Using defau